In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
import pandas as pd
from sklearn.metrics import accuracy_score


In [46]:
df=pd.read_csv('/content/drive/MyDrive/Thub_netlist.csv')
df=df.dropna(axis=1,how="any",thresh=None)
df=df.drop(columns=['50'])
df = df.dropna()
df = df.replace("'Trojan Infected'",1)
df = df.replace("'Trojan Free'",0)
df

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1,2,3,4,5,6,7,8,9,...,39,41,42,43,44,45,46,47,48,49
0,Number of ports,Number of nets,Number of cells,Number of combinational cells,Number of sequential cells,Number of macros/black boxes,Number of buf/inv,Number of references,Combinational area,Buf/Inv area,...,Sequential Leakage Power,Combinational Internal Power,Combinational Switching Power,Combinational Leakage Power,Combinational Total Power,Total Internal Power,Total Switching Power,Total Leakage Power,Total Total Power,Label
1,26623,198177,172368,164960,6850,0,19504,22,173046.832,10376.1281,...,0,18253,32696,5135800,56082,45560,38947,5677500,90182,0
2,27289,198776,172486,164991,6934,0,19504,2,173081.678,10376.1281,...,0,18260,32707,5136600,56101,45820,38963,5685000,90465,1
3,27545,198893,172468,164973,6934,0,19504,2,173066.25,10376.1281,...,0,18257,32707,5136200,56097,45851,38965,5685000,90498,1
4,28033,198562,172369,164960,6850,0,19504,1,173046.832,10376.1281,...,0,18250,32693,5135800,56076,45553,38942,5677500,90170,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,359,8376,6866,0,6866,0,0,27,0,0,...,0,0,726.6476,0,726.6476,0,726.6476,0,726.6476,1
904,359,8373,6863,0,6863,0,0,26,0,0,...,0,0,726.4709,0,726.4709,0,726.4709,0,726.4709,1
905,359,8374,6864,0,6864,0,0,26,0,0,...,0,0,726.5132,0,726.5132,0,726.5132,0,726.5132,1
906,359,8378,6868,0,6868,0,0,25,0,0,...,0,0,726.6879,0,726.6879,0,726.6879,0,726.6879,1


In [47]:
import numpy as np
arr=np.array(df.values)
X=arr[1:,0:48]
X=X.astype('float')
Y=(arr[1:,47]).astype('float')

In [48]:
X.shape

(907, 48)

In [49]:
#Y

In [50]:
#Splitting the dataset into training and test set.  
from sklearn.model_selection import train_test_split  
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size= 0.2, random_state=0)

In [51]:
from sklearn.feature_selection import chi2

In [52]:
 from sklearn.feature_selection import SelectKBest

In [53]:
chi2_selector = SelectKBest(chi2, k=11)

In [54]:
chi2_selector = SelectKBest(chi2, k=11)
X_kbest = chi2_selector.fit_transform(X, Y)
print(X_kbest)

print('Original number of features:', X.shape)
print('Reduced number of features:', X_kbest.shape)

[[1.981770e+05 1.723680e+05 1.649600e+05 ... 5.608200e+04 5.677500e+06
  9.018200e+04]
 [1.987760e+05 1.724860e+05 1.649910e+05 ... 5.610100e+04 5.685000e+06
  9.046500e+04]
 [1.988930e+05 1.724680e+05 1.649730e+05 ... 5.609700e+04 5.685000e+06
  9.049800e+04]
 ...
 [8.374000e+03 6.864000e+03 0.000000e+00 ... 7.265132e+02 0.000000e+00
  7.265132e+02]
 [8.378000e+03 6.868000e+03 0.000000e+00 ... 7.266879e+02 0.000000e+00
  7.266879e+02]
 [8.369000e+03 6.859000e+03 0.000000e+00 ... 7.263525e+02 0.000000e+00
  7.263525e+02]]
Original number of features: (907, 48)
Reduced number of features: (907, 11)


In [55]:
X_kbest

array([[1.981770e+05, 1.723680e+05, 1.649600e+05, ..., 5.608200e+04,
        5.677500e+06, 9.018200e+04],
       [1.987760e+05, 1.724860e+05, 1.649910e+05, ..., 5.610100e+04,
        5.685000e+06, 9.046500e+04],
       [1.988930e+05, 1.724680e+05, 1.649730e+05, ..., 5.609700e+04,
        5.685000e+06, 9.049800e+04],
       ...,
       [8.374000e+03, 6.864000e+03, 0.000000e+00, ..., 7.265132e+02,
        0.000000e+00, 7.265132e+02],
       [8.378000e+03, 6.868000e+03, 0.000000e+00, ..., 7.266879e+02,
        0.000000e+00, 7.266879e+02],
       [8.369000e+03, 6.859000e+03, 0.000000e+00, ..., 7.263525e+02,
        0.000000e+00, 7.263525e+02]])

In [56]:
X_kbest

array([[1.981770e+05, 1.723680e+05, 1.649600e+05, ..., 5.608200e+04,
        5.677500e+06, 9.018200e+04],
       [1.987760e+05, 1.724860e+05, 1.649910e+05, ..., 5.610100e+04,
        5.685000e+06, 9.046500e+04],
       [1.988930e+05, 1.724680e+05, 1.649730e+05, ..., 5.609700e+04,
        5.685000e+06, 9.049800e+04],
       ...,
       [8.374000e+03, 6.864000e+03, 0.000000e+00, ..., 7.265132e+02,
        0.000000e+00, 7.265132e+02],
       [8.378000e+03, 6.868000e+03, 0.000000e+00, ..., 7.266879e+02,
        0.000000e+00, 7.266879e+02],
       [8.369000e+03, 6.859000e+03, 0.000000e+00, ..., 7.263525e+02,
        0.000000e+00, 7.263525e+02]])

In [57]:
Y.shape

(907,)

In [58]:
#Splitting the dataset into training and test set.  
from sklearn.model_selection import train_test_split  
X_kbest_train, X_kbest_test, Y_train, Y_test= train_test_split(X_kbest, Y, test_size= 0.2, random_state=0)

Select from Model

Random Forest

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
clf = RandomForestClassifier(n_estimators =10)

clf.fit(X_kbest_train, Y_train)


Y_pred = clf.predict(X_kbest_test)
print(accuracy_score(Y_test,Y_pred))

from sklearn import metrics
print(metrics.classification_report(Y_test, Y_pred))


0.9835164835164835
              precision    recall  f1-score   support

         0.0       1.00      0.40      0.57         5
         1.0       0.98      1.00      0.99       177

    accuracy                           0.98       182
   macro avg       0.99      0.70      0.78       182
weighted avg       0.98      0.98      0.98       182



SVM

In [62]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(X_kbest_train, Y_train) 

SVC(random_state=0)

In [63]:
Y_pred= classifier.predict(X_kbest_test) 

In [64]:
print(accuracy_score(Y_test, Y_pred)*100)

from sklearn import metrics
print(metrics.classification_report(Y_test, Y_pred))

97.25274725274726
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         5
         1.0       0.97      1.00      0.99       177

    accuracy                           0.97       182
   macro avg       0.49      0.50      0.49       182
weighted avg       0.95      0.97      0.96       182



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Logistic Regression

In [65]:
from sklearn.linear_model import LogisticRegression


In [66]:

clf = LogisticRegression(random_state=0, solver='liblinear', max_iter=300,multi_class='ovr')
clf.fit(X_kbest_train, Y_train)

    
Y_pred = clf.predict(X_kbest_test)

accuracy = 100 * accuracy_score(Y_test, Y_pred)

print(accuracy)

from sklearn import metrics
print(metrics.classification_report(Y_test, Y_pred))


96.15384615384616
              precision    recall  f1-score   support

         0.0       0.25      0.20      0.22         5
         1.0       0.98      0.98      0.98       177

    accuracy                           0.96       182
   macro avg       0.61      0.59      0.60       182
weighted avg       0.96      0.96      0.96       182



KNN

In [67]:
from sklearn.neighbors import KNeighborsClassifier


In [68]:

          
clf = KNeighborsClassifier(n_neighbors=10)
    
clf.fit(X_kbest_train, Y_train)

Y_pred = clf.predict(X_kbest_test)

accuracy = 100 * accuracy_score(Y_test, Y_pred)


print(accuracy)

from sklearn import metrics
print(metrics.classification_report(Y_test, Y_pred))

97.25274725274726
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         5
         1.0       0.97      1.00      0.99       177

    accuracy                           0.97       182
   macro avg       0.49      0.50      0.49       182
weighted avg       0.95      0.97      0.96       182



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XGBOOST CLASSIFIER

In [69]:
from xgboost import XGBClassifier


In [70]:

   
clf = XGBClassifier(n_estimators=20)
    
clf.fit(X_kbest_train, Y_train)
   
    
Y_pred = clf.predict(X_kbest_test)

  
accuracy = 100 * accuracy_score(Y_test, Y_pred)

print(accuracy)

from sklearn import metrics
print(metrics.classification_report(Y_test, Y_pred))

96.15384615384616
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         5
         1.0       0.97      0.99      0.98       177

    accuracy                           0.96       182
   macro avg       0.49      0.49      0.49       182
weighted avg       0.95      0.96      0.95       182



Decision Tree Classifier

In [71]:
from sklearn.tree import DecisionTreeClassifier 

clf = DecisionTreeClassifier()

clf = clf.fit(X_kbest_train,Y_train)

Y_pred = clf.predict(X_kbest_test)


In [72]:
Accuracy= 100 * accuracy_score(Y_test, Y_pred)
print(Accuracy)

from sklearn import metrics
print(metrics.classification_report(Y_test, Y_pred))


98.9010989010989
              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75         5
         1.0       0.99      1.00      0.99       177

    accuracy                           0.99       182
   macro avg       0.99      0.80      0.87       182
weighted avg       0.99      0.99      0.99       182



Gaussian Naive Baye's

In [73]:
from sklearn.naive_bayes import GaussianNB


# instantiate the model
gnb = GaussianNB()


# fit the model
gnb.fit(X_kbest_train, Y_train)

Y_pred = gnb.predict(X_kbest_test)
accuracy = 100 * (accuracy_score(Y_test, Y_pred))
print(accuracy)

from sklearn import metrics
print(metrics.classification_report(Y_test, Y_pred))

94.5054945054945
              precision    recall  f1-score   support

         0.0       0.14      0.20      0.17         5
         1.0       0.98      0.97      0.97       177

    accuracy                           0.95       182
   macro avg       0.56      0.58      0.57       182
weighted avg       0.95      0.95      0.95       182



Ensemble Learning

In [74]:
from sklearn.ensemble import VotingClassifier

model_1 = LogisticRegression(random_state=0, solver='liblinear', max_iter=300, multi_class='ovr')
model_2 = XGBClassifier(n_estimators=20)
model_3 = RandomForestClassifier(n_estimators=5, max_depth=5, random_state=1)
model_4 = KNeighborsClassifier(n_neighbors=3)
model_5 = SVC(kernel="rbf", C=10, gamma=1)
final_model = VotingClassifier(estimators=[('lr', model_1), 
                                             ('xgb', model_2), 
                                             ('rf', model_3), 
                                             ('knn',model_4),
                                             ('svm', model_5)], voting='hard')
final_model.fit(X_kbest_train, Y_train)


pred_final = final_model.predict(X_kbest_test)

accuracy = 100 * accuracy_score(Y_test, pred_final)
  
print("Accuracy = %.2f" % (accuracy))

from sklearn import metrics
print(metrics.classification_report(Y_test, pred_final))



Accuracy = 97.25
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         5
         1.0       0.97      1.00      0.99       177

    accuracy                           0.97       182
   macro avg       0.49      0.50      0.49       182
weighted avg       0.95      0.97      0.96       182



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
